In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

### (b) Gurobi

In [2]:
# Read excel sheets and transform them into lists and matrices
facility=pd.read_excel('OR109-2_hw02_data.xlsx','Facility',index_col = 0,header=1)
facility=facility.iloc[:,:9]
facility.rename(columns = {'Unnamed: 9':'Population'}, inplace = True)

In [3]:
facility

,1,2,3,4,5,6,7,8,Population
1,0,3,4,6,8,9,8,10,40
2,3,0,5,4,8,6,12,9,30
3,4,5,0,2,2,3,5,7,35
4,6,4,2,0,3,2,5,4,20
5,8,8,2,3,0,2,2,4,15
6,9,6,3,2,2,0,3,2,50
7,8,12,5,5,2,3,0,2,45
8,10,9,7,4,4,2,2,0,60


In [4]:
D=facility.iloc[:,:8].values.tolist()
P=facility.iloc[:,8].values.tolist()

In [5]:
# decide the n and m parameters
n=8
m=3

In [6]:
hw2 = Model("hw2")

Academic license - for non-commercial use only - expires 2021-05-30
Using license file /Users/terrylu/gurobi.lic


In [7]:
# add variables as a list 定義X 
x = []
for i in range(n):
    x.append(hw2.addVar(lb = 0, vtype = GRB.BINARY, name = "x_" +  str(i+1) ))

In [8]:
# add variables as a list 定義y
y = []
for i in range(n):
    y.append([])
    for j in range(n):
        y[i].append(hw2.addVar(lb = 0, vtype = GRB.BINARY, name = "y_" +  str(i+1) +'_'+ str(j+1) ))

In [9]:
# define w as the maximum population-weighted firefighting time among all districts
w = hw2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w" )

In [10]:
# setting the objective function 
hw2.setObjective( w , GRB.MINIMIZE) 

In [11]:
hw2.addConstrs(( 
                quicksum(y[i][j] for i in range(n) ) ==1
                for j in range(n) 
                ), " y_ij equal 1" )

hw2.addConstrs(( 
                quicksum(x[i] for i in range(n) ) <= m 
                for k in range(1)
                ), "x smaller than m" )

hw2.addConstrs(( 
                quicksum(y[i][j] for j in range(n) ) <= n*x[i]
                for i in range(n) 
                ), " y_ij smaller than x_i" )

hw2.addConstrs(( 
                w >= (P[j] * quicksum(D[i][j]*y[i][j] for i in range(n))  )
                for j in range(n) 
                ), " w is maximum" )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>}

In [12]:
hw2.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 25 rows, 73 columns and 208 nonzeros
Model fingerprint: 0xfef31b4f
Variable types: 1 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 320.0000000
Presolve removed 0 rows and 9 columns
Presolve time: 0.00s
Presolved: 25 rows, 64 columns, 181 nonzeros
Variable types: 0 continuous, 64 integer (63 binary)

Root relaxation: objective 0.000000e+00, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    6  320.00000    0.00000   100%     -    0s
H    0     0                     180.0000000    0.00000   100%   

In [13]:
print("Result:")
for i in range(n):
    print(x[i].varName, '=', x[i].x)

Result:
x_1 = 1.0
x_2 = -0.0
x_3 = -0.0
x_4 = 1.0
x_5 = -0.0
x_6 = -0.0
x_7 = -0.0
x_8 = 1.0


In [14]:
df_y=pd.DataFrame(y)
df_y=df_y.applymap(lambda y : y.x )
df_y.columns=range(1,9)
df_y.index=range(1,9)

In [15]:
df_y

,1,2,3,4,5,6,7,8
1,1.0,1.0,-0.0,-0.0,-0.0,0.0,0.0,0.0
2,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0
3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0
4,-0.0,-0.0,1.0,1.0,-0.0,0.0,-0.0,-0.0
5,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
6,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
7,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
8,0.0,-0.0,-0.0,-0.0,1.0,1.0,1.0,1.0


In [16]:
print("z* =", hw2.objVal)    # print objective value

z* = 100.0


### (c) heuristic algorithm 

In [27]:
# decide the n and m parameters
n=8
m=3

In [28]:
D

[[0, 3, 4, 6, 8, 9, 8, 10],
 [3, 0, 5, 4, 8, 6, 12, 9],
 [4, 5, 0, 2, 2, 3, 5, 7],
 [6, 4, 2, 0, 3, 2, 5, 4],
 [8, 8, 2, 3, 0, 2, 2, 4],
 [9, 6, 3, 2, 2, 0, 3, 2],
 [8, 12, 5, 5, 2, 3, 0, 2],
 [10, 9, 7, 4, 4, 2, 2, 0]]

In [29]:
P

[40, 30, 35, 20, 15, 50, 45, 60]

In [30]:
# define decision variable x_true 
x_true=[0]*n

In [31]:
x_true

[0, 0, 0, 0, 0, 0, 0, 0]

In [32]:
# define decision variable y
y=[]
for i in range(n):
    y.append([])
    for j in range(n):
        y[i].append(0)

In [33]:
y

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [34]:
for num_ambulance in range(m): #要依序加入ambulance 直到有m臺 ambulance
    for_loop_minimum=[] #之後用來排序 計算每個iteration哪個地區的maximum population-weighted firefighting times最小
    for i in range(n): #在n個地區依序嘗試加入ambulance到第i個地區來計算firefighting times
        if x_true[i]==1: #如果已經真實加入ambulance則不用再嘗試加入
            continue
        x_for_decide_minimum=x_true.copy() #在iteration之中的x 因為我們會嘗試依序加入ambulance至各地區 所以跟x_true區別
        x_for_decide_minimum[i]=1 #這個iteration之中嘗試加入ambulance到第i個地區
        print("this round we try to add:",x_for_decide_minimum) #印出這回合加入i地區以後x的樣子
        d_df=pd.DataFrame(D)
        d_df["x"]=x_for_decide_minimum
        decide_min=d_df.groupby('x').idxmin().loc[1] #只考慮有ambulance地區來計算對於其他地區 哪一個有ambulance地區是最近的
        y=[] #將y重新恢復原始值
        for ii in range(n):
            y.append([])
            for jj in range(n):
                y[ii].append(0)
                
        #根據ambulance地區最近的邏輯，給予y值
        for k in range(n):
            y[decide_min.values[k]][decide_min.index[k]]=1
        
        #加入第i個地區以及該地區加入ambulance的話 maximum population-weighted firefighting times是多少
        for_loop_minimum.append([i,max(np.sum(np.array(y)*np.array(D),axis=0)*P)]) 
        
    #排序在這一個iteration：所有地區以及該地區加入ambulance的話 maximum population-weighted firefighting times 由小到大排序
    for_loop_minimum=pd.DataFrame(for_loop_minimum).sort_values(1)
    for_loop_minimum=for_loop_minimum.values.tolist()
    
    #根據排序值找到這一個iteration要將哪個地區真的加入ambulance
    for i in range(len(for_loop_minimum)):
        small_index=for_loop_minimum[i][0]
        if x_true[small_index]!=1: #要目前沒有ambulance才能被加入
            x_true[small_index]=1
            last_objective_minimum=for_loop_minimum[i][1] #紀錄目前最低的maximum population-weighted firefighting times
            print("ambulance added at District"+str(small_index+1)+" maximum times is "+str(last_objective_minimum))
            break
        else:
            continue
    print("after this iteration x_true become:",x_true) #印出這回合iteration後 x真實樣子
    print()
    
#最後印出加入ambulance至m台以後 目前最低的maximum population-weighted firefighting times 也就是objective value 是多少
print('final objective value is',last_objective_minimum) 

this round we try to add: [1, 0, 0, 0, 0, 0, 0, 0]
this round we try to add: [0, 1, 0, 0, 0, 0, 0, 0]
this round we try to add: [0, 0, 1, 0, 0, 0, 0, 0]
this round we try to add: [0, 0, 0, 1, 0, 0, 0, 0]
this round we try to add: [0, 0, 0, 0, 1, 0, 0, 0]
this round we try to add: [0, 0, 0, 0, 0, 1, 0, 0]
this round we try to add: [0, 0, 0, 0, 0, 0, 1, 0]
this round we try to add: [0, 0, 0, 0, 0, 0, 0, 1]
ambulance added at District4 maximum population-weighted firefighting times is 240
after this iteration x_true become: [0, 0, 0, 1, 0, 0, 0, 0]

this round we try to add: [1, 0, 0, 1, 0, 0, 0, 0]
this round we try to add: [0, 1, 0, 1, 0, 0, 0, 0]
this round we try to add: [0, 0, 1, 1, 0, 0, 0, 0]
this round we try to add: [0, 0, 0, 1, 1, 0, 0, 0]
this round we try to add: [0, 0, 0, 1, 0, 1, 0, 0]
this round we try to add: [0, 0, 0, 1, 0, 0, 1, 0]
this round we try to add: [0, 0, 0, 1, 0, 0, 0, 1]
ambulance added at District1 maximum population-weighted firefighting times is 240
after t

In [36]:
x_true

[1, 0, 0, 1, 0, 0, 0, 1]

In [37]:
y

[[1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1]]

In [38]:
print('final objective value is',last_objective_minimum)

final objective value is 100
